In [143]:
import requests
import re
import pandas as pd
from PIL import Image
import hashlib

import asyncio
import aiohttp
from time import time

In [362]:
async def load_image(url, session, tag):
    async with session.get('http://www.humansnotinvited.com/' + url, allow_redirects=True) as response:
        data = await response.read()
    hsh = hashlib.md5(data).hexdigest()
    d[tag] = d.get(tag, {})
    d[tag][hsh] = d[tag].get(hsh, 0) + 1

In [307]:
async def load_pages(session):
    async with session.get('http://www.humansnotinvited.com/', allow_redirects=True) as response:
        data = await response.read()
    images = find_all_images(data)
    tag = find_tag(data)
    tasks = []
    for i in images:
        tasks.append(asyncio.create_task(load_image(i, session, tag)))
    await asyncio.gather(*tasks)

In [308]:
def find_all_images(content):
    return re.findall('captcha/image.php\\?image_name=.*?&id=.', str(content))

In [309]:
def find_tag(content):
    return re.findall('value="(.*?)" name="category"', str(content))[0]

In [337]:
async def main():
    n = 1
    tasks = []
    async with aiohttp.ClientSession() as session:
        for i in range(n):
            if not i % 25:
                print(i)
            await load_pages(session)

# Start collecting the data for the algorithm 

In [338]:
t0 = time()
await main()
time() - t0

0


2.4895269870758057

In [353]:
df = pd.DataFrame(d)
df.shape

(217, 24)

# Save / Load results from / to the file

In [354]:
df.to_csv('nothuman.csv')

In [267]:
df = pd.read_csv('nothuman.csv', index_col=0)

# Type here your html code for http://www.humansnotinvited.com/ page

In [350]:
content = """
<div class="header">
            <p>Select all squares with <strong>star trek</strong></p>
            <input type="hidden" value="star_trek" name="category" />
          </div>

          <div class="content">
            <div class="captcha-image" data-token="$1$m4m0mb1q$EDIrQjvIDte/hYJjR89.F0" data-id="2"><img src="captcha/image.php?image_name=$1$m4m0mb1q$EDIrQjvIDte/hYJjR89.F0&id=2" alt="" /></div><div class="captcha-image" data-token="$1$jsqPPWXs$XgF0H7RtYYwtpSQAYMgwn0" data-id="9"><img src="captcha/image.php?image_name=$1$jsqPPWXs$XgF0H7RtYYwtpSQAYMgwn0&id=9" alt="" /></div><div class="captcha-image" data-token="$1$yrNKx9O8$j/ioQRdn2SwzJck1xVfTl0" data-id="7"><img src="captcha/image.php?image_name=$1$yrNKx9O8$j/ioQRdn2SwzJck1xVfTl0&id=7" alt="" /></div><div class="captcha-image" data-token="$1$23flc1Rj$Cy2JGeb7Z15uK3E3ZLl3a/" data-id="8"><img src="captcha/image.php?image_name=$1$23flc1Rj$Cy2JGeb7Z15uK3E3ZLl3a/&id=8" alt="" /></div><div class="captcha-image" data-token="$1$UQ5x0oN7$qgEAm5IGMntWBAxSyIk0r1" data-id="6"><img src="captcha/image.php?image_name=$1$UQ5x0oN7$qgEAm5IGMntWBAxSyIk0r1&id=6" alt="" /></div><div class="captcha-image" data-token="$1$JL7.Jl7k$8jrJv.0CLfbqIT2YF0AXs1" data-id="3"><img src="captcha/image.php?image_name=$1$JL7.Jl7k$8jrJv.0CLfbqIT2YF0AXs1&id=3" alt="" /></div><div class="captcha-image" data-token="$1$yT0VqnxU$pE8RFD6IxBNmOLckcGn8f." data-id="6"><img src="captcha/image.php?image_name=$1$yT0VqnxU$pE8RFD6IxBNmOLckcGn8f.&id=6" alt="" /></div><div class="captcha-image" data-token="$1$dsP3V8Ed$PdBwvhke0DspvfMSW2sau/" data-id="4"><img src="captcha/image.php?image_name=$1$dsP3V8Ed$PdBwvhke0DspvfMSW2sau/&id=4" alt="" /></div><div class="captcha-image" data-token="$1$4/GbOZQn$PDAvPJyFLyPm6rCluw8n9." data-id="2"><img src="captcha/image.php?image_name=$1$4/GbOZQn$PDAvPJyFLyPm6rCluw8n9.&id=2" alt="" /></div>          </div>
        </div>
"""

In [351]:
q = []
new_images, new_tag = find_all_images(content), find_tag(content)
async with aiohttp.ClientSession() as session:
    for img in new_images:
        async with session.get('http://www.humansnotinvited.com/' + img, allow_redirects=True) as response:
            data = await response.read()
        hsh = hashlib.md5(data).hexdigest()
        q.append(hsh)

# Select the cells with the numbers below
<span>1|2|3</span><br/><span>4|5|6</span><br/><span>7|8|9</span>

In [352]:
answ = df.loc[q, [new_tag]] > 8
for i in range(len(answ)):
    if answ.iloc[i][new_tag]:
        print(i + 1)

1
2
4
